# Neural Networks

In [1]:
import os
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

'2'

In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
from scipy import stats

/Users/melo/miniconda3/envs/my_tf_env/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data

In [3]:

# Load the CSV file
data = pd.read_csv("BC-Data-Set.csv")

In [4]:
# Convert the date column to a datetime object
data['date'] = pd.to_datetime(data['date'])

# Remove any missing values
data = data.dropna()

# Set the date column as the index of the DataFrame
data = data.set_index('date')

In [5]:
seed = 42

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data, data.BC, test_size=0.15, random_state=seed, shuffle=True)
x_train = x_train.drop(columns=['BC'])
x_test = x_test.drop(columns=['BC'])

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=seed, shuffle=True)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [8]:
# TODO: split, preprocess, make sequences

In [9]:
x_train

array([[-0.73291434,  1.55477006,  3.60894021, ...,  1.95560752,
        -0.89521109,  0.69813568],
       [-0.85503739, -0.08410553, -0.35847915, ..., -0.37050872,
        -0.48279646,  0.2517087 ],
       [-0.38903519, -0.40824082, -0.54120237, ...,  0.15400768,
        -1.71157723, -1.90387708],
       ...,
       [-0.89772   , -0.72621189, -0.91761219, ..., -0.39331378,
         0.94389125, -0.81840302],
       [-0.48238073,  0.10302261, -0.19784883, ...,  1.11182025,
         1.35890992, -0.49238002],
       [-0.7281987 , -0.953701  , -1.27475302, ..., -0.48453403,
        -1.55972922, -2.74749276]])

In [10]:
x_train.shape

(3050, 12)

In [11]:
import tensorflow as tf
from tensorflow import keras

## Neural Network (non-LSTM)

In [12]:
HIDDEN_DIM = 256
inputs = tf.keras.layers.Input(shape=[x_train.shape[-1]])

layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(inputs)
layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(layer)
layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(layer)
layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(layer)

layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(layer)

predictions = keras.layers.Dense(1, activation=None)(layer)

model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),
            loss=tf.losses.mean_squared_error,
            metrics=[tf.metrics.mean_squared_error],
        )


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [13]:
batch_size = 32
epochs = 2000
logs = model.fit(
    x_train, y_train,
    batch_size=batch_size, epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_mean_squared_error', mode='min', patience=30,  restore_best_weights=True)
                            ],
)

Epoch 1/2000
96/96 [==============================] - 2s 14ms/step - loss: 0.6867 - mean_squared_error: 0.6867 - val_loss: 0.4742 - val_mean_squared_error: 0.4742
Epoch 2/2000
96/96 [==============================] - 1s 9ms/step - loss: 0.4431 - mean_squared_error: 0.4431 - val_loss: 0.5747 - val_mean_squared_error: 0.5747
Epoch 3/2000
96/96 [==============================] - 1s 8ms/step - loss: 0.3690 - mean_squared_error: 0.3690 - val_loss: 0.4572 - val_mean_squared_error: 0.4572
Epoch 4/2000
96/96 [==============================] - 1s 8ms/step - loss: 0.3182 - mean_squared_error: 0.3182 - val_loss: 0.3761 - val_mean_squared_error: 0.3761
Epoch 5/2000
96/96 [==============================] - 1s 8ms/step - loss: 0.2795 - mean_squared_error: 0.2795 - val_loss: 0.3797 - val_mean_squared_error: 0.3797
Epoch 6/2000
96/96 [==============================] - 1s 9ms/step - loss: 0.2782 - mean_squared_error: 0.2782 - val_loss: 0.3786 - val_mean_squared_error: 0.3786
Epoch 7/2000
96/96 [=======

## LSTM

In [14]:
# TODO
def build_sequences(df, window=200, stride=200):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    for id in df['id'].unique():
        # Take only meaningful features
        temp = df[df['id'] == id][['x_axis','y_axis','z_axis']].values
        # Save the label
        label = df[df['id'] == id]['activity'].values[0]
        print(str(id), "  ", str(label))
        # Compute padding length
        padding_len = window - len(temp)%window
        # Create padding and concatenate it
        padding = np.zeros((padding_len,3), dtype='float64')
        temp = np.concatenate((temp,padding))
        # Build features windows with their corresponging labels
        idx = 0
        while idx+window <= len(temp):
            dataset.append(temp[idx:idx+window])
            labels.append(label)
            idx += stride
    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [16]:
def build_sequences(df, target_labels=['BC'], window=200, stride=200, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float32')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
X_train, y_train = build_sequences(x_train, target_labels, window, stride, telescope)
X_test, y_test = build_sequences(x_test, target_labels, window, stride, telescope)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [15]:
SEQUENCE_LENGTH = 8
SEQUENCE_DIM = x_train.shape[-1]
RNN_CELL_DIM = 8
HIDDEN_DIM = 8
sequences = tf.keras.layers.Input(shape=[SEQUENCE_LENGTH, SEQUENCE_DIM])

layer = keras.layers.LSTM(RNN_CELL_DIM, return_sequences=True)(sequences)

layer = keras.layers.Dense(HIDDEN_DIM, activation='relu')(layer)

predictions = keras.layers.Dense(1, activation=None)(layer)

model = tf.keras.Model(inputs=sequences, outputs=predictions)
model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=tf.losses.mean_squared_error,
            metrics=[tf.metrics.mean_squared_error],
        )
